# Kinetica Vectorstore API

>[Kinetica](https://www.kinetica.com/) is a database with integrated support for vector similarity search

It supports:
- exact and approximate nearest neighbor search
- L2 distance, inner product, and cosine distance

This notebook shows how to use the Kinetica vector store (`Kinetica`).

This needs an instance of Kinetica which can easily be setup using the instructions given here - [installation instruction](https://www.kinetica.com/developer-edition/).

In [1]:
# install package
%pip install --upgrade --quiet langchain-kinetica
%pip install --upgrade --quiet  langchain-openai tiktoken ipykernel

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Environment Setup

You will need the following environment variables defined. 

> Note: If you are using VSCode you can put them in a `.env` file at the root of your project and restart your notbook kernel.

Kinetica Connection:
* `KINETICA_URL`: Database connection URL
* `KINETICA_USER`: Database user
* `KINETICA_PASSWD`: Secure password.

OpeAI Connection:
* `OPENAI_API_KEY`

Your `.env` file might look like this:
```
KINETICA_URL = http://172.1.1.1:9191/
KINETICA_USER = admin
KINETICA_PASSWD = password
OPENAI_API_KEY = sk-XXX
```

In [1]:
# Load imports
from langchain_community.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain_kinetica.vectorstores import (
    KineticaVectorStore,
    KineticaSettings,
)
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [3]:
loader = TextLoader("./state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
KINETICA_SETTINGS = KineticaSettings()

## Similarity Search with Euclidean Distance (Default)

In [4]:
# The Kinetica Module will try to create a table with the name of the collection.
# So, make sure that the collection name is unique and the user has the permission to create a table.

COLLECTION_NAME = "state_of_the_union_test"

db = KineticaVectorStore.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    config=KINETICA_SETTINGS,
)

2024-05-15 02:32:48.825 INFO     [GPUdb] Connected to Kinetica! (host=http://172.31.33.30:9191 api=7.2.0.7 server=7.2.0.6)


In [5]:
query = "What did the president say about Ketanji Brown Jackson"
docs_with_score = db.similarity_search_with_score(query)

In [6]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.6076346039772034
Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
--------------------------------------------------------------------------------
-----------------------

## Maximal Marginal Relevance Search (MMR)
Maximal marginal relevance optimizes for similarity to query AND diversity among selected documents.

In [7]:
docs_with_score = db.max_marginal_relevance_search_with_score(query)

In [8]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.6076346039772034
Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
--------------------------------------------------------------------------------
-----------------------

## Working with vectorstore

Above, we created a vectorstore from scratch. However, often times we want to work with an existing vectorstore.
In order to do that, we can initialize it directly.

In [9]:
store = KineticaVectorStore(
    collection_name=COLLECTION_NAME,
    config=KINETICA_SETTINGS,
    embedding_function=embeddings,
)

2024-05-15 02:32:49.433 INFO     [GPUdb] Connected to Kinetica! (host=http://172.31.33.30:9191 api=7.2.0.7 server=7.2.0.6)


### Add documents
We can add documents to the existing vectorstore.

In [10]:
store.add_documents([Document(page_content="foo")])

['3b78fa93-f06e-4af7-86da-31d1cc820ca5']

In [11]:
docs_with_score = db.similarity_search_with_score("foo")

In [12]:
docs_with_score[0]

(Document(page_content='foo'), 0.0)

In [13]:
docs_with_score[1]

(Document(page_content='A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. \n\nWe can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  \n\nWe’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.  \n\nWe’re putting in place dedicated immigration judges so families fleeing persecution and violence can have their cases heard faster. \n\nWe’re securing commitments and supporting partners in South and Central America to host more refugees and secure their own borders.', metadata={'source': './state_of_the_union.txt'})

### Overriding a vectorstore

If you have an existing collection, you override it by doing `from_documents` and setting `pre_delete_collection` = True

In [14]:
db = KineticaVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    config=KINETICA_SETTINGS,
    pre_delete_collection=True,
)

2024-05-15 02:32:50.786 INFO     [GPUdb] Connected to Kinetica! (host=http://172.31.33.30:9191 api=7.2.0.7 server=7.2.0.6)


In [15]:
docs_with_score = db.similarity_search_with_score("foo")

In [16]:
docs_with_score[0]

(Document(page_content='A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. \n\nWe can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  \n\nWe’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.  \n\nWe’re putting in place dedicated immigration judges so families fleeing persecution and violence can have their cases heard faster. \n\nWe’re securing commitments and supporting partners in South and Central America to host more refugees and secure their own borders.', metadata={'source': './state_of_the_union.txt'})

### Using a VectorStore as a Retriever

In [17]:
retriever = store.as_retriever()

In [18]:
print(retriever)

tags=['KineticaVectorStore', 'OpenAIEmbeddings'] vectorstore=<langchain_kinetica.vectorstores.KineticaVectorStore object at 0x7fc4a5252880>
